In [ ]:
import SimpleITK as sitk
import numpy as np
import os

def convert_dicom_to_nifti(input_folder):
    reader = sitk.ImageSeriesReader()
    dicom_series = reader.GetGDCMSeriesFileNames(input_folder)
    reader.SetFileNames(dicom_series)
    image = reader.Execute()
    return image

def resample_image_3d(itk_image, new_spacing=(0.5, 0.5, 5.0)):
    original_spacing = itk_image.GetSpacing()
    original_size = itk_image.GetSize()
    
    out_spacing = new_spacing
    
    out_size = [
        int(np.round(original_size[0] * original_spacing[0] / new_spacing[0])),
        int(np.round(original_size[1] * original_spacing[1] / new_spacing[1])),
        int(np.round(original_size[2] * original_spacing[2] / new_spacing[2]))
    ]
    
    resample = sitk.ResampleImageFilter()
    resample.SetOutputSpacing(out_spacing)
    resample.SetSize(out_size)
    resample.SetOutputDirection(itk_image.GetDirection())
    resample.SetOutputOrigin(itk_image.GetOrigin())
    resample.SetTransform(sitk.Transform())
    resample.SetDefaultPixelValue(itk_image.GetPixelIDValue())
    resample.SetInterpolator(sitk.sitkLinear)
    
    return resample.Execute(itk_image)

def has_dicom_files(directory):
    try:
        reader = sitk.ImageSeriesReader()
        dicom_series = reader.GetGDCMSeriesFileNames(directory)
        return len(dicom_series) > 0
    except:
        return False

def is_leaf_directory(directory):
    for item in os.listdir(directory):
        if os.path.isdir(os.path.join(directory, item)):
            return False
    return True

def get_unique_filename(base_path, filename):
    if not os.path.exists(os.path.join(base_path, filename)):
        return filename
    
    name, ext = os.path.splitext(filename)
    counter = 1
    while os.path.exists(os.path.join(base_path, f"{name}_{counter}{ext}")):
        counter += 1
    return f"{name}_{counter}{ext}"

def process_directory(input_folder, output_folder, target_spacing=(0.5, 0.5, 5.0)):
    if not os.path.exists(output_folder):
        os.makedirs(output_folder)
    
    processed_count = 0
    failed_count = 0
    failed_dirs = []
    processed_dirs = []
    
    dicom_dirs = []
    
    for root, dirs, files in os.walk(input_folder):
        if is_leaf_directory(root) and has_dicom_files(root):
            dicom_dirs.append(root)
    
    for idx, dicom_dir in enumerate(dicom_dirs, 1):
        try:
            relative_path = os.path.relpath(dicom_dir, input_folder)
            
            safe_filename = relative_path.replace(os.sep, '_').replace(' ', '_')
            output_filename = f'{safe_filename}_resampled.nii.gz'
            
            if len(output_filename) > 100:
                path_parts = relative_path.split(os.sep)
                if len(path_parts) >= 2:
                    safe_filename = f"{path_parts[-2]}_{path_parts[-1]}"
                else:
                    safe_filename = path_parts[-1] if path_parts else f"scan_{idx}"
                output_filename = f'{safe_filename}_resampled.nii.gz'
            
            output_filename = get_unique_filename(output_folder, output_filename)
            output_file_path = os.path.join(output_folder, output_filename)
            
            original_img = convert_dicom_to_nifti(dicom_dir)
            resampled_img = resample_image_3d(original_img, new_spacing=target_spacing)
            sitk.WriteImage(resampled_img, output_file_path)
            
            processed_count += 1
            processed_dirs.append({
                'input': dicom_dir,
                'output': output_filename,
                'original_spacing': original_img.GetSpacing(),
                'original_size': original_img.GetSize(),
                'resampled_spacing': resampled_img.GetSpacing(),
                'resampled_size': resampled_img.GetSize()
            })
            
        except Exception as e:
            failed_count += 1
            failed_dirs.append({'dir': dicom_dir, 'error': str(e)})
    
    print(f'Processing completed: {processed_count}/{len(dicom_dirs)} succeeded')
    
    if failed_count > 0:
        print(f'Failed: {failed_count} directories')
    
    log_file = os.path.join(output_folder, 'processing_log.txt')
    with open(log_file, 'w', encoding='utf-8') as f:
        f.write(f'DICOM Resampling Processing Log\n')
        f.write(f'{"="*70}\n')
        f.write(f'Input directory: {input_folder}\n')
        f.write(f'Output directory: {output_folder}\n')
        f.write(f'Target voxel spacing: {target_spacing} mm³\n')
        f.write(f'Successfully processed: {processed_count} DICOM series\n')
        if failed_count > 0:
            f.write(f'Failed processing: {failed_count} directories\n')
        f.write(f'\nList of successfully processed files:\n')
        f.write('-' * 70 + '\n')
        
        for i, info in enumerate(processed_dirs, 1):
            f.write(f'\n[{i}] {info["output"]}\n')
            f.write(f'    Source directory: {info["input"]}\n')
            f.write(f'    Original size: {info["original_size"]}\n')
            f.write(f'    Original spacing: {info["original_spacing"]} mm\n')
            f.write(f'    Resampled size: {info["resampled_size"]}\n')
            f.write(f'    Resampled spacing: {info["resampled_spacing"]} mm\n')
        
        if failed_dirs:
            f.write(f'\n\nFailed directories:\n')
            f.write('-' * 70 + '\n')
            for fail_info in failed_dirs:
                f.write(f'  Directory: {fail_info["dir"]}\n')
                f.write(f'  Error: {fail_info["error"]}\n\n')

def main():
    input_folder = r'Path to your DICOM files'
    output_folder = r'Path to save resampled files'
    
    target_spacing = (0.5, 0.5, 5.0)
    
    if not os.path.exists(input_folder):
        print(f'Error: Input directory does not exist')
        return
    
    process_directory(input_folder, output_folder, target_spacing=target_spacing)

if __name__ == "__main__":
    main()